# Part VIII: Asynchronous Programming

## Chapter 17: Mastering Async/Await

FastAPI's exceptional performance stems from its async-native architecture. However, simply using `async def` doesn't automatically make code non-blocking. Understanding when to use synchronous versus asynchronous functions, identifying blocking operations, and properly offloading CPU-intensive work is crucial for building high-performance applications. This chapter demystifies Python's async model in the context of FastAPI, ensuring you leverage the full power of the event loop.

---

### 17.1 `def` vs `async def`: When to Use Which

FastAPI supports both synchronous (`def`) and asynchronous (`async def`) path operation functions. Choosing incorrectly can block the event loop and destroy performance, or introduce unnecessary complexity.

#### The Event Loop and Thread Pool Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│           FastAPI Request Handling Architecture                  │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Incoming Request                                                │
│       │                                                          │
│       ▼                                                          │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │              Main Thread (Event Loop)                    │    │
│  │  ┌─────────────────────────────────────────────────────┐ │    │
│  │  │ async def endpoints                                │ │    │
│  │  │ • Database queries (asyncpg, aiomysql)            │ │    │
│  │  │ • HTTP client (httpx, aiohttp)                    │ │    │
│  │  │ • WebSocket operations                             │ │    │
│  │  │ • FastAPI dependencies (async)                     │ │    │
│  │  │                                                    │ │    │
│  │  │  ↓ Await points allow other requests to run        │ │    │
│  │  └─────────────────────────────────────────────────────┘ │    │
│  │                          │                               │    │
│  │  ┌───────────────────────▼─────────────────────────────┐ │    │
│  │  │ def endpoints (run in thread pool)                 │ │    │
│  │  │ • CPU-intensive calculations                       │ │    │
│  │  │ • Blocking I/O (pandas, requests, time.sleep)      │ │    │
│  │  │ • File operations (open(), read())                 │ │    │
│  │  │                                                    │ │    │
│  │  │  ↓ Runs in separate thread, loop continues         │ │    │
│  │  └─────────────────────────────────────────────────────┘ │    │
│  └─────────────────────────────────────────────────────────┘    │
│                                                                  │
│  Thread Pool ( Starlette manages this )                         │
│  ┌─────────┐ ┌─────────┐ ┌─────────┐                           │
│  │ Thread 1│ │ Thread 2│ │ Thread 3│ ...                       │
│  │ (def)   │ │ (def)   │ │ (def)   │                           │
│  └─────────┘ └─────────┘ └─────────┘                           │
│                                                                  │
│  Rule of Thumb:                                                  │
│  • Use async def for I/O-bound async libraries                   │
│  • Use def for CPU-bound or blocking synchronous code             │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

#### Decision Framework

```python
# async_vs_sync.py - Decision examples

from fastapi import FastAPI, Depends
import asyncio
import time
import httpx
import requests

app = FastAPI()

# ═════════════════════════════════════════════════════════════════
# USE async def WHEN:
# Working with async libraries (database, HTTP clients)
# ═════════════════════════════════════════════════════════════════

@app.get("/db-async")
async def get_from_database():
    """
    CORRECT: async def with async database driver.
    
    The 'await' yields control back to the event loop,
    allowing other requests to be processed while waiting
    for the database response.
    """
    # SQLAlchemy async session
    result = await db.execute(select(User).where(User.id == 1))
    # During the await above, the event loop handles other requests
    return result.scalar_one()


@app.get("/http-async")
async def fetch_external_api():
    """
    CORRECT: async def with async HTTP client (httpx).
    
    Non-blocking HTTP request.
    """
    async with httpx.AsyncClient() as client:
        response = await client.get("https://api.example.com/data")
        # Event loop continues during network wait
    return response.json()


# ═════════════════════════════════════════════════════════════════
# USE def WHEN:
# CPU-intensive work or blocking synchronous libraries
# ═════════════════════════════════════════════════════════════════

@app.get("/cpu-intensive")
def cpu_bound_calculation(n: int = 1000000):
    """
    CORRECT: def for CPU-intensive work.
    
    FastAPI runs this in a thread pool automatically.
    If this were async def, it would block the event loop!
    """
    total = 0
    for i in range(n):
        total += i ** 2  # CPU-bound calculation
    return {"result": total}


@app.get("/blocking-http")
def blocking_external_api():
    """
    CORRECT: def with blocking library (requests).
    
    requests.get() is synchronous and would block the event loop
    if used in async def. Using def puts it in a thread pool.
    """
    # This blocks the thread, but not the event loop
    response = requests.get("https://api.example.com/slow", timeout=30)
    return response.json()


@app.get("/file-read")
def read_large_file():
    """
    CORRECT: def for file I/O.
    
    open() and read() are blocking system calls.
    """
    with open("large_file.txt", "r") as f:
        content = f.read()  # Blocks thread, not event loop
    return {"content_length": len(content)}


# ═════════════════════════════════════════════════════════════════
# INCORRECT PATTERNS (Anti-patterns)
# ═════════════════════════════════════════════════════════════════

@app.get("/wrong-db")
async def bad_database_access():
    """
    WRONG: async def with synchronous database driver.
    
    psycopg2 (sync) inside async def blocks the event loop.
    Use asyncpg with SQLAlchemy async instead.
    """
    import psycopg2  # Synchronous library!
    conn = psycopg2.connect(DATABASE_URL)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")  # BLOCKS EVENT LOOP!
    return cursor.fetchall()


@app.get("/wrong-cpu")
async def bad_cpu_bound():
    """
    WRONG: async def with CPU-intensive work.
    
    No await means no yield point. The event loop is blocked
    and cannot handle other requests during calculation.
    """
    total = 0
    for i in range(10000000):  # Blocks for seconds!
        total += i
    return {"total": total}


@app.get("/wrong-sleep")
async def bad_sleep():
    """
    WRONG: async def with blocking time.sleep.
    
    Use asyncio.sleep instead for async functions.
    """
    time.sleep(5)  # BLOCKS EVENT LOOP - BAD!
    return {"message": "Done"}


@app.get("/correct-sleep")
async def good_sleep():
    """
    CORRECT: asyncio.sleep yields control.
    """
    await asyncio.sleep(5)  # Non-blocking - event loop continues
    return {"message": "Done"}
```

**Decision Matrix:**

| Operation Type | Library Examples | Use `def` or `async def` | Why |
|---|---|---|---|
| Database queries | asyncpg, aiomysql, SQLAlchemy 2.0 async | `async def` | Native async support, non-blocking I/O |
| Database queries | psycopg2, pymysql (sync) | `def` | Blocking I/O, run in thread pool |
| HTTP requests | httpx, aiohttp | `async def` | Native async, connection pooling |
| HTTP requests | requests, urllib | `def` | Blocking, run in thread pool |
| CPU-intensive | pandas, numpy, calculations | `def` | Releases GIL or runs in thread |
| File operations | Standard open(), pathlib | `def` | System calls block |
| File operations | aiofiles | `async def` | Async wrapper for files |
| Caching/Redis | redis-py (async) | `async def` | Async redis client |

---

### 17.2 Blocking vs Non-Blocking: Identifying Performance Bottlenecks

Not all `await` calls are equal, and not all "async" code is truly non-blocking. Understanding what happens under the hood helps identify bottlenecks.

#### Identifying Blocking Operations

```python
# blocking_detection.py - Identifying what blocks

import asyncio
import time
from concurrent.futures import ThreadPoolExecutor
from fastapi import FastAPI, Request
import psutil
import os

app = FastAPI()

# Middleware to detect blocking operations
@app.middleware("http")
async def detect_blocking(request: Request, call_next):
    """
    Middleware that warns if endpoint blocks for too long.
    
    Useful for development to catch accidentally blocking code.
    """
    start_time = time.time()
    
    # Create task to check if we're blocking
    loop = asyncio.get_event_loop()
    warning_threshold = 0.1  # 100ms
    
    async def check_blocking():
        """Check if event loop is responsive."""
        await asyncio.sleep(warning_threshold)
        # If we get here, the loop isn't blocked
        return False
    
    # Run check and endpoint concurrently
    endpoint_task = asyncio.create_task(call_next(request))
    check_task = asyncio.create_task(check_blocking())
    
    done, pending = await asyncio.wait(
        [endpoint_task, check_task],
        return_when=asyncio.FIRST_COMPLETED
    )
    
    duration = time.time() - start_time
    
    # If endpoint finished before check, it might be blocking
    if endpoint_task in done and check_task in pending:
        check_task.cancel()
        if duration > warning_threshold:
            process = psutil.Process(os.getpid())
            print(f"⚠️  WARNING: Potentially blocking endpoint "
                  f"{request.url.path} took {duration:.2f}s")
    
    response = await endpoint_task
    return response


# Examples of blocking vs non-blocking

@app.get("/non-blocking-async")
async def truly_non_blocking():
    """
    Truly non-blocking: await yields control immediately.
    
    The event loop can process hundreds of other requests
    during this 2-second wait.
    """
    print("Starting non-blocking wait...")
    await asyncio.sleep(2)  # Immediately yields control
    print("Finished non-blocking wait")
    return {"type": "non-blocking"}


@app.get("/blocking-sync")
def blocking_function():
    """
    Blocking: occupies a thread but releases event loop.
    
    FastAPI runs this in a thread pool. The main thread
    (event loop) remains free to accept new connections.
    However, this consumes one thread from the pool.
    """
    print("Starting blocking sleep...")
    time.sleep(2)  # Blocks this thread only
    print("Finished blocking sleep")
    return {"type": "blocking-thread"}


@app.get("/cpu-intensive-def")
def cpu_bound():
    """
    CPU-intensive in def: Runs in thread, may release GIL.
    
    If the calculation releases the GIL (numpy, pandas),
    other threads can run. If not (pure Python), it hogs
    the thread but doesn't block the event loop.
    """
    # Pure Python - hogs CPU but in separate thread
    total = sum(i**2 for i in range(10_000_000))
    return {"result": total}


@app.get("/cpu-intensive-async")
async def cpu_bound_bad():
    """
    CPU-intensive in async: BLOCKS EVENT LOOP!
    
    This freezes the entire application for all users
    while the calculation runs. Never do this.
    """
    # BAD: Blocks the event loop
    total = sum(i**2 for i in range(10_000_000))
    return {"result": total}


# Testing concurrency
async def make_request(client, endpoint):
    """Helper to time requests."""
    start = time.time()
    response = await client.get(endpoint)
    duration = time.time() - start
    return endpoint, duration, response.status_code


@app.get("/test-concurrency")
async def test_concurrency():
    """
    Test to demonstrate the difference between blocking
    and non-blocking endpoints.
    """
    from httpx import AsyncClient
    
    async with AsyncClient(app=app, base_url="http://test") as client:
        # Time 5 concurrent requests to non-blocking endpoint
        tasks = [make_request(client, "/non-blocking-async") for _ in range(5)]
        results = await asyncio.gather(*tasks)
        
        # If truly non-blocking, all should complete in ~2 seconds total
        # If blocking, would take ~10 seconds (2s * 5 sequential)
        
        total_time = max(r[1] for r in results)
        return {
            "endpoint": "non-blocking-async",
            "concurrent_requests": 5,
            "max_time": total_time,
            "results": [{"endpoint": r[0], "time": r[1]} for r in results]
        }
```

---

### 17.3 Running Blocking Code: `run_in_threadpool`

When you must use blocking libraries (pandas, sklearn, requests) inside `async def` endpoints, you need to explicitly run them in a thread pool to avoid blocking the event loop.

#### Using run_in_threadpool

```python
# threadpool.py - Offloading blocking code

from fastapi import FastAPI, Depends
from starlette.concurrency import run_in_threadpool
import asyncio
import time
import pandas as pd  # Blocking library
import numpy as np   # Blocking library (releases GIL)
import requests      # Blocking library

app = FastAPI()

# ═════════════════════════════════════════════════════════════════
# Pattern 1: Simple offloading with run_in_threadpool
# ═════════════════════════════════════════════════════════════════

@app.post("/process-csv")
async def process_csv_async(file_path: str):
    """
    Run pandas (blocking) in thread pool.
    
    run_in_threadpool schedules the function in Starlette's
    thread pool executor and awaits the result without
    blocking the event loop.
    """
    # This runs in a thread pool
    df = await run_in_threadpool(pd.read_csv, file_path)
    
    # Process in thread pool too
    result = await run_in_threadpool(lambda: df.describe().to_dict())
    
    return {"statistics": result}


async def heavy_calculation(data: list):
    """CPU-intensive function to offload."""
    # Simulate heavy work
    time.sleep(2)  # Blocking!
    return sum(x ** 2 for x in data)


@app.post("/calculate")
async def calculate_endpoint(data: list[int]):
    """
    Offload CPU work to thread pool.
    
    Without run_in_threadpool, this would freeze the API.
    """
    # Offload to thread
    result = await run_in_threadpool(heavy_calculation, data)
    
    # Event loop is free during calculation
    return {"result": result}


# ═════════════════════════════════════════════════════════════════
# Pattern 2: Multiple concurrent blocking operations
# ═════════════════════════════════════════════════════════════════

@app.post("/fetch-multiple")
async def fetch_multiple_urls(urls: list[str]):
    """
    Fetch multiple URLs concurrently using thread pool.
    
    Even though requests is synchronous, we can run multiple
    instances concurrently in the thread pool.
    """
    # Create tasks for concurrent execution
    tasks = [
        run_in_threadpool(requests.get, url, timeout=30)
        for url in urls
    ]
    
    # Gather results concurrently
    responses = await asyncio.gather(*tasks)
    
    return {
        "results": [
            {"url": url, "status": r.status_code}
            for url, r in zip(urls, responses)
        ]
    }


# ═════════════════════════════════════════════════════════════════
# Pattern 3: Custom ThreadPoolExecutor for heavy workloads
# ═════════════════════════════════════════════════════════════════

from concurrent.futures import ThreadPoolExecutor
import functools

# Custom executor for CPU-intensive tasks
cpu_executor = ThreadPoolExecutor(max_workers=4, thread_name_prefix="cpu_worker")

@app.post("/ml-predict")
async def ml_prediction(data: dict):
    """
    Machine learning inference (CPU-bound).
    
    Uses custom executor to isolate heavy CPU work.
    """
    # Load model (blocking I/O)
    model = await run_in_threadpool(load_model, "model.pkl")
    
    # Prediction (CPU-intensive)
    # Use custom executor for better control
    loop = asyncio.get_event_loop()
    result = await loop.run_in_executor(
        cpu_executor,
        functools.partial(model.predict, data)
    )
    
    return {"prediction": result.tolist()}


# ═════════════════════════════════════════════════════════════════
# Pattern 4: Database operations with sync drivers
# ═════════════════════════════════════════════════════════════════

import psycopg2  # Synchronous database driver

def sync_db_query(user_id: int):
    """Synchronous database operation."""
    conn = psycopg2.connect(DATABASE_URL)
    try:
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM users WHERE id = %s", (user_id,))
            return cur.fetchone()
    finally:
        conn.close()


@app.get("/users-sync/{user_id}")
async def get_user_sync_driver(user_id: int):
    """
    Use synchronous database driver in async endpoint.
    
    Not recommended for production (use asyncpg instead),
    but useful for legacy code migration.
    """
    user = await run_in_threadpool(sync_db_query, user_id)
    if not user:
        raise HTTPException(status_code=404, detail="User not found")
    return {"user": user}


# ═════════════════════════════════════════════════════════════════
# Pattern 5: File I/O with async facade
# ═════════════════════════════════════════════════════════════════

@app.post("/upload-process")
async def upload_and_process(file: UploadFile):
    """
    Read file and process with blocking library.
    """
    # Read file content (async)
    content = await file.read()
    
    # Process in thread pool (blocking)
    def process_content(data: bytes):
        # Simulate processing with pandas/numpy
        import io
        df = pd.read_csv(io.BytesIO(data))
        return df.head(100).to_json()
    
    result = await run_in_threadpool(process_content, content)
    
    return {"processed": result}


# ═════════════════════════════════════════════════════════════════
# Cleanup on shutdown
# ═════════════════════════════════════════════════════════════════

@app.on_event("shutdown")
async def shutdown_event():
    """Cleanup thread pools on application shutdown."""
    cpu_executor.shutdown(wait=True)
```

**Key Points about run_in_threadpool:**

1. **Starlette's Default Pool**: FastAPI/Starlette maintains a default thread pool. You don't need to create one for simple use cases.
2. **Context Matters**: `run_in_threadpool` is for `async def` endpoints. If you use `def`, FastAPI already runs it in a thread pool automatically.
3. **GIL Considerations**: For pure Python CPU work, threads don't provide true parallelism (due to GIL). For numpy/pandas operations that release the GIL, threads do provide parallelism.
4. **Process Pool**: For heavy pure-Python CPU work, consider `ProcessPoolExecutor` instead of `ThreadPoolExecutor` to bypass the GIL.

---

### 17.4 Background Tasks: `BackgroundTasks` for Fire-and-Forget

Some operations shouldn't delay the HTTP response. Sending emails, processing images, or logging can run after the response is sent using FastAPI's `BackgroundTasks`.

#### Background Tasks Implementation

```python
# background_tasks.py - Fire-and-forget operations

from fastapi import FastAPI, BackgroundTasks, Depends
from fastapi.mail import FastMail, MessageSchema
from sqlalchemy.ext.asyncio import AsyncSession
import asyncio
import logging

app = FastAPI()
logger = logging.getLogger(__name__)

# ═════════════════════════════════════════════════════════════════
# Basic Background Tasks
# ═════════════════════════════════════════════════════════════════

def send_email_sync(email: str, subject: str, body: str):
    """
    Synchronous email sending function.
    
    Runs in background thread after response is sent.
    """
    # Simulate slow email sending
    import time
    time.sleep(2)
    logger.info(f"Email sent to {email}: {subject}")


@app.post("/register")
async def register_user(
    user_data: UserCreate,
    background_tasks: BackgroundTasks,
    db: AsyncSession = Depends(get_db)
):
    """
    Register user and send welcome email in background.
    
    The response returns immediately (201 Created).
    The email sends after the response is sent.
    """
    # Create user (blocking, but fast)
    user = await create_user(db, user_data)
    
    # Add email task to background
    background_tasks.add_task(
        send_email_sync,
        email=user.email,
        subject="Welcome!",
        body=f"Hello {user.username}, welcome to our platform!"
    )
    
    # Return immediately - email sends after this
    return {"id": user.id, "message": "User created"}


# ═════════════════════════════════════════════════════════════════
# Async Background Tasks
# ═════════════════════════════════════════════════════════════════

async def process_image_async(image_path: str, user_id: str):
    """
    Async background task.
    
    Can use await because BackgroundTasks supports both
    sync and async functions.
    """
    # Simulate async image processing
    await asyncio.sleep(5)
    
    # Update database
    async with AsyncSessionLocal() as db:
        await db.execute(
            update(User)
            .where(User.id == user_id)
            .values(avatar_processed=True)
        )
        await db.commit()
    
    logger.info(f"Image processed for user {user_id}")


@app.post("/upload-avatar")
async def upload_avatar(
    file: UploadFile,
    background_tasks: BackgroundTasks,
    current_user: CurrentUser
):
    """
    Upload avatar and process in background.
    
    User gets immediate confirmation while image
    is resized/optimized asynchronously.
    """
    # Save file immediately (fast)
    file_path = f"/uploads/{current_user.id}_{file.filename}"
    with open(file_path, "wb") as f:
        content = await file.read()
        f.write(content)
    
    # Process in background
    background_tasks.add_task(
        process_image_async,
        image_path=file_path,
        user_id=current_user.id
    )
    
    return {
        "message": "Avatar uploaded, processing in background",
        "path": file_path
    }


# ═════════════════════════════════════════════════════════════════
# Chaining Background Tasks
# ═════════════════════════════════════════════════════════════════

def log_activity_sync(user_id: str, action: str):
    """Log to external analytics (sync)."""
    import requests
    requests.post(
        "https://analytics.example.com/track",
        json={"user_id": user_id, "action": action}
    )

async def update_cache_async(user_id: str):
    """Update cache (async)."""
    await redis.set(f"user:{user_id}", "updated", ex=3600)

@app.post("/perform-action")
async def perform_action(
    action: str,
    background_tasks: BackgroundTasks,
    current_user: CurrentUser
):
    """
    Chain multiple background tasks.
    
    Tasks run sequentially in the order added.
    """
    # Primary action
    result = await do_action(current_user.id, action)
    
    # Add multiple background tasks
    background_tasks.add_task(
        log_activity_sync,
        current_user.id,
        action
    )
    
    background_tasks.add_task(
        update_cache_async,
        current_user.id
    )
    
    background_tasks.add_task(
        notify_followers,
        current_user.id,
        action
    )
    
    return {"status": "success", "result": result}


# ═════════════════════════════════════════════════════════════════
# Error Handling in Background Tasks
# ═════════════════════════════════════════════════════════════════

async def risky_background_task(data: str):
    """
    Background task with error handling.
    
    Errors in background tasks don't affect the HTTP response
    but should be logged and monitored.
    """
    try:
        # Risky operation
        result = await external_api_call(data)
        await save_to_db(result)
    except Exception as e:
        # Log error - response already sent
        logger.error(f"Background task failed: {e}", exc_info=True)
        
        # Could send alert to monitoring
        await send_alert_to_ops(f"Task failed for data: {data}")


@app.post("/process-risky")
async def process_risky(
    data: str,
    background_tasks: BackgroundTasks
):
    """
    Fire risky operation in background.
    
    User gets 202 Accepted immediately.
    Success/failure handled separately.
    """
    background_tasks.add_task(risky_background_task, data)
    
    return {
        "status": "accepted",
        "message": "Processing in background"
    }


# ═════════════════════════════════════════════════════════════════
# Task Dependencies (Advanced)
# ═════════════════════════════════════════════════════════════════

from typing import Callable

def get_task_function(task_type: str) -> Callable:
    """Factory for background task functions."""
    tasks = {
        "email": send_email_sync,
        "process_image": process_image_async,
        "log": log_activity_sync
    }
    return tasks.get(task_type, lambda x: None)

@app.post("/generic-task/{task_type}")
async def generic_task(
    task_type: str,
    payload: dict,
    background_tasks: BackgroundTasks
):
    """Dynamic background task selection."""
    task_func = get_task_function(task_type)
    
    background_tasks.add_task(task_func, **payload)
    
    return {"task": task_type, "status": "queued"}


# ═════════════════════════════════════════════════════════════════
# Important Limitations
# ═════════════════════════════════════════════════════════════════

"""
IMPORTANT LIMITATIONS:

1. No Result Access: You cannot get the return value of a
   background task. If you need results, use Celery or RQ.

2. No Retry: If a background task fails, it's gone.
   For critical operations, use a proper task queue.

3. Process Bound: If the process restarts, pending
   background tasks are lost. Not for critical data.

4. Sequential Execution: Tasks added to BackgroundTasks
   run sequentially, not in parallel.

For production-heavy workloads, use Celery, RQ, or ARQ instead.
"""
```

**Background Tasks vs Celery:**

| Feature | BackgroundTasks | Celery/ARQ |
|---------|----------------|------------|
| **Complexity** | Built-in, simple | Requires broker (Redis/RabbitMQ) |
| **Persistence** | In-memory only | Persistent queue |
| **Retries** | No | Yes, with exponential backoff |
| **Result storage** | No | Yes, with result backend |
| **Monitoring** | Logs only | Web UI, detailed metrics |
| **Distributed** | Single process | Multi-worker, multi-server |
| **Use case** | Simple post-processing | Heavy async workloads |

---

### Summary

In this chapter, you mastered Python's async model in the context of FastAPI:

1. **`def` vs `async def`**: Use `async def` with native async libraries (databases, HTTP clients) to yield control to the event loop. Use `def` for CPU-bound work or blocking synchronous libraries (pandas, requests), letting FastAPI run them in thread pools automatically.

2. **Blocking Identification**: Recognized that `time.sleep`, synchronous file I/O, and pure Python CPU calculations block the event loop if used in `async def`. Learned to detect blocking code through timing analysis and middleware.

3. **`run_in_threadpool`**: Used `starlette.concurrency.run_in_threadpool` to explicitly offload blocking operations (database queries with sync drivers, ML inference, file processing) to thread pools within async endpoints, preventing event loop blockage.

4. **Background Tasks**: Implemented fire-and-forget operations using FastAPI's `BackgroundTasks` for non-critical post-processing (emails, logging, cache warming) that shouldn't delay HTTP responses.

**Performance Checklist:**
- Profile endpoints to detect blocking code
- Use `async def` only with truly async libraries
- Offload blocking code with `run_in_threadpool`
- Use `def` for CPU-intensive endpoints
- Use BackgroundTasks for non-critical post-processing
- For heavy workloads, migrate to Celery/ARQ

---

### What's Next?

**Chapter 18: WebSockets** will cover:
- **WebSocket Basics**: Establishing persistent, full-duplex connections between client and server for real-time communication
- **Handling Messages**: Implementing bidirectional message sending and receiving with proper connection state management
- **Broadcasting**: Broadcasting messages to multiple connected clients using connection managers and pub/sub patterns (Redis)
- **WebSockets vs SSE**: Choosing between WebSockets (bidirectional) and Server-Sent Events (unidirectional) based on use case requirements

This next chapter enables you to build real-time features like chat applications, live notifications, and collaborative editing.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../7. testing_and_quality_assurance/16. code_quality.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='18. websockets.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
